In [29]:
from google.cloud import bigquery

In [30]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/content/multichannel-attribution-dc955547de5a.json"

In [31]:
client = bigquery.Client()

In [32]:
dataset_ref = client.dataset('google_analytics_sample', project='bigquery-public-data')

In [33]:
print(dataset_ref)

DatasetReference('bigquery-public-data', 'google_analytics_sample')


In [34]:

# Fetch the dataset
dataset = client.get_dataset(dataset_ref)

In [35]:
# Get list of tables
tables = client.list_tables(dataset)
[x.table_id for x in tables]

['ga_sessions_20160801',
 'ga_sessions_20160802',
 'ga_sessions_20160803',
 'ga_sessions_20160804',
 'ga_sessions_20160805',
 'ga_sessions_20160806',
 'ga_sessions_20160807',
 'ga_sessions_20160808',
 'ga_sessions_20160809',
 'ga_sessions_20160810',
 'ga_sessions_20160811',
 'ga_sessions_20160812',
 'ga_sessions_20160813',
 'ga_sessions_20160814',
 'ga_sessions_20160815',
 'ga_sessions_20160816',
 'ga_sessions_20160817',
 'ga_sessions_20160818',
 'ga_sessions_20160819',
 'ga_sessions_20160820',
 'ga_sessions_20160821',
 'ga_sessions_20160822',
 'ga_sessions_20160823',
 'ga_sessions_20160824',
 'ga_sessions_20160825',
 'ga_sessions_20160826',
 'ga_sessions_20160827',
 'ga_sessions_20160828',
 'ga_sessions_20160829',
 'ga_sessions_20160830',
 'ga_sessions_20160831',
 'ga_sessions_20160901',
 'ga_sessions_20160902',
 'ga_sessions_20160903',
 'ga_sessions_20160904',
 'ga_sessions_20160905',
 'ga_sessions_20160906',
 'ga_sessions_20160907',
 'ga_sessions_20160908',
 'ga_sessions_20160909',


In [36]:
# Fetch the first table as sample table
df_ref = dataset_ref.table('ga_sessions_20160801')
df = client.get_table(df_ref)

# Use table dataset to list the rows
df_rows = client.list_rows(df).to_dataframe()
df_rows.head()

,visitorId,visitNumber,visitId,visitStartTime,date,totals,trafficSource,device,geoNetwork,customDimensions,hits,fullVisitorId,userId,channelGrouping,socialEngagementType
0,None,1,1470117657,1470117657,20160801,"{'visits': 1, 'hits': 3, 'pageviews': 3, 'time...","{'referralPath': '/yt/about/', 'campaign': '(n...","{'browser': 'Internet Explorer', 'browserVersi...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 23, 'minu...",7194065619159478122,None,Social,Not Socially Engaged
1,None,151,1470083489,1470083489,20160801,"{'visits': 1, 'hits': 3, 'pageviews': 3, 'time...","{'referralPath': '/yt/about/', 'campaign': '(n...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 13, 'minu...",8159312408158297118,None,Social,Not Socially Engaged
2,None,1,1470052694,1470052694,20160801,"{'visits': 1, 'hits': 4, 'pageviews': 3, 'time...","{'referralPath': '/yt/about/', 'campaign': '(n...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Asia', 'subContinent': 'Southea...",[],"[{'hitNumber': 1, 'time': 0, 'hour': 4, 'minut...",9236304747882138291,None,Social,Not Socially Engaged
3,None,1,1470061879,1470061879,20160801,"{'visits': 1, 'hits': 4, 'pageviews': 4, 'time...","{'referralPath': '/yt/about/', 'campaign': '(n...","{'browser': 'Firefox', 'browserVersion': 'not ...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 7, 'minut...",1792676004815023069,None,Social,Not Socially Engaged
4,None,1,1470090830,1470090830,20160801,"{'visits': 1, 'hits': 4, 'pageviews': 2, 'time...","{'referralPath': '/yt/about/', 'campaign': '(n...","{'browser': 'Internet Explorer', 'browserVersi...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 15, 'minu...",7305625498291809599,None,Social,Not Socially Engaged


In [38]:
df_rows.shape

(1711, 15)

In [40]:
# create a table of metrics by channel

query = """
    SELECT
        channelGrouping as channel,
        SUM(totals.totalTransactionRevenue)/1000000 AS revenue,
        SUM(totals.transactions) AS transactions,
        COUNT(DISTINCT fullVisitorId) AS users,
        SUM(totals.visits) AS sessions,
        SUM(totals.pageviews) AS pageviews
    FROM 
        `bigquery-public-data.google_analytics_sample.ga_sessions_*`
    WHERE
        _TABLE_SUFFIX BETWEEN '20161001' AND '20161230'
        AND totals.totalTransactionRevenue IS NOT NULL
    GROUP BY
        1
    ORDER BY
        2 DESC
"""
safe_query_job = client.query(query)
df_3 = safe_query_job.to_dataframe()
df_3.head(5)

,channel,revenue,transactions,users,sessions,pageviews
0,Referral,182578.60,1540,1331,1470,42829
1,Direct,128141.06,568,456,539,16132
2,Organic Search,93376.88,991,905,956,28247
3,Paid Search,14028.37,164,153,160,4277
4,Display,4728.45,34,34,34,1044
